In [ ]:
import pandas as pd
import os
file_path = "/content/API Call Dataset.csv"
df = pd.read_csv(file_path)
print("Initial Data:")
print(df.head())
print("\nColumn Names:", df.columns)

api_column = "API Code"
api_call_counts = df[api_column].value_counts().nlargest(4)
top_apis = api_call_counts.index

print("\nTop 4 APIs based on the number of calls:")
print(api_call_counts)
output_directory = os.path.dirname(file_path)
for api in top_apis:
    output_path = os.path.join(output_directory, f"{api}_data.csv")
    api_df = df[df[api_column] == api]
    api_df.to_csv(output_path, index=False)
    print(f"Data for {api} saved to {output_path}")


Initial Data:
  API Code      Time of Call
0       A1  01-02-2025 00:00
1       A3  01-02-2025 00:17
2       A3  01-02-2025 00:19
3       A8  01-02-2025 00:37
4       A8  01-02-2025 00:47

Column Names: Index(['API Code', 'Time of Call'], dtype='object')

Top 4 APIs based on the number of calls:
API Code
A9    2451
A2    2438
A7    2410
A4    2402
Name: count, dtype: int64
Data for A9 saved to /content/A9_data.csv
Data for A2 saved to /content/A2_data.csv
Data for A7 saved to /content/A7_data.csv
Data for A4 saved to /content/A4_data.csv


In [ ]:
!pip install tsai fastai torch pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tsai.all import *
from fastai.tabular.all import *
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

def get_models(c_in=2, c_out=1, seq_len=24):
    """Return dictionary of all five models using direct PyTorch implementations"""

    # LSTM - using nn.LSTM directly
    lstm = nn.Sequential(
        nn.LSTM(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )

    # GRU - using nn.GRU directly
    gru = nn.Sequential(
        nn.GRU(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )

    # MLP - using nn.Sequential directly
    mlp = nn.Sequential(
        nn.Flatten(),
        nn.Linear(seq_len * c_in, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, c_out)
    )

    # FCN and ResNet from tsai
    fcn = FCN(c_in=c_in, c_out=c_out)
    resnet = ResNet(c_in=c_in, c_out=c_out)

    return {
        'LSTM': lstm,
        'GRU': gru,
        'MLP': mlp,
        'FCN': fcn,
        'ResNet': resnet
    }

def model_forward(model, x, model_name):
    """Handle forward pass for all model types"""
    if model_name in ['LSTM', 'GRU']:
        x = x.transpose(1, 2)  # Change to (batch, seq_len, features)
        if isinstance(model, nn.Sequential):
            output, _ = model[0](x)
            return model[1](output[:, -1, :])
    elif model_name == 'MLP':
        # MLP expects flattened input
        return model(x)
    else:
        # FCN and ResNet expect (batch, features, seq_len)
        return model(x)

def prepare_data(df, seq_length=24, split_ratio=0.8):
    """Prepare data for models with correct dimensions"""
    # Sort by timestamp to ensure temporal order
    df = df.sort_values('Time of Call')

    # Calculate time differences in seconds and normalize
    time_diffs = df['Time of Call'].diff().dt.total_seconds()
    time_diffs = time_diffs.fillna(time_diffs.mean())

    # Normalize time differences
    time_diffs = (time_diffs - time_diffs.mean()) / time_diffs.std()

    # Create sequences
    X, y = [], []
    for i in range(len(df) - seq_length):
        # Create sequence with 2 features: time_diff and count
        sequence = np.column_stack([
            time_diffs.iloc[i:i+seq_length],
            np.ones(seq_length)  # Event indicator
        ])
        X.append(sequence)
        y.append([1])  # Predicting next event

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)

    # For FCN and ResNet, shape should be (n_samples, n_features, seq_length)
    X = X.transpose(0, 2, 1)

    # Convert to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    # Create splits
    train_size = int(len(X) * split_ratio)
    splits = (list(range(train_size)), list(range(train_size, len(X))))

    # Create TSai datasets
    tfms = [None, [TSRegression()]]
    dsets = TSDatasets(X, y, splits=splits, tfms=tfms)

    batch_size = min(64, len(X))
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_size=batch_size)

    return dls, df['Time of Call'].iloc[-1]

def make_single_prediction(model, df, model_name):
    """Make prediction for next event"""
    # Prepare last sequence
    time_diffs = df['Time of Call'].diff().dt.total_seconds().fillna(0)

    # Normalize time differences
    time_diffs = (time_diffs - time_diffs.mean()) / time_diffs.std()

    last_sequence = np.column_stack([
        time_diffs.iloc[-24:].values,
        np.ones(24)
    ])

    # Reshape to (batch, features, seq_length) for FCN/ResNet
    last_sequence = last_sequence.transpose().reshape(1, 2, 24)

    model.eval()
    with torch.no_grad():
        x = torch.tensor(last_sequence, dtype=torch.float32)
        pred = model_forward(model, x, model_name)
        pred_value = max(0, float(pred[0] if len(pred.shape) > 1 else pred))

    return pred_value

def calculate_mae(y_true, y_pred):
    """Calculate Mean Absolute Error"""
    return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))

def main():
    # Load the original dataset
    original_file = "API Call Dataset.csv"  # Update path as needed
    df_original = pd.read_csv(original_file)

    # Get the top 4 APIs
    api_counts = df_original['API Code'].value_counts().nlargest(4)
    top_apis = api_counts.index

    # Dictionary to store results
    results = {}
    last_timestamps = {}
    mae_scores = {}

    for api in top_apis:
        print(f"\nProcessing {api}...")

        # Load API-specific data
        file_path = f"{api}_data.csv"
        df = pd.read_csv(file_path)

        # Convert timestamp to datetime
        df['Time of Call'] = pd.to_datetime(df['Time of Call'], format='%d-%m-%Y %H:%M')

        # Prepare data
        dls, last_timestamp = prepare_data(df)
        last_timestamps[api] = last_timestamp

        # Get models
        models = get_models()
        api_predictions = {}
        api_mae = {}

        # Train each model and make predictions
        for model_name, model in models.items():
            print(f"Training {model_name}...")

            # Create learner with custom forward pass
            class ModelWrapper(nn.Module):
                def __init__(self, base_model, model_name):
                    super().__init__()
                    self.base_model = base_model
                    self.model_name = model_name

                def forward(self, x):
                    return model_forward(self.base_model, x, self.model_name)

            wrapped_model = ModelWrapper(model, model_name)
            learn = Learner(dls, wrapped_model, metrics=[rmse])

            # Train model
            learn.fit_one_cycle(10, lr_max=1e-3)

            # Make prediction
            prediction = make_single_prediction(model, df, model_name)
            api_predictions[model_name] = prediction

            # Calculate MAE on validation set
            valid_dl = dls.valid
            all_preds = []
            all_targets = []

            # Collect predictions and targets
            model.eval()
            with torch.no_grad():
                for batch in valid_dl:
                    x, y = batch
                    pred = model_forward(model, x, model_name)
                    all_preds.extend(pred.numpy().flatten())
                    all_targets.extend(y.numpy().flatten())

            mae = calculate_mae(all_targets, all_preds)
            api_mae[model_name] = mae

            print(f"{model_name} prediction: {prediction:.2f}, MAE: {mae:.4f}")

        results[api] = api_predictions
        mae_scores[api] = api_mae

    # Create formatted output
    print("\nLast timestamps for each API:")
    for api, timestamp in last_timestamps.items():
        print(f"{api}: {timestamp}")

    print("\nPredicted number of events for next occurrence:")
    predictions_df = pd.DataFrame(results)
    print(predictions_df.round(2))

    print("\nMAE scores for each model:")
    mae_df = pd.DataFrame(mae_scores)
    print(mae_df.round(4))

    # Save results to CSV
    predictions_df.to_csv('model_predictions.csv')
    mae_df.to_csv('model_mae_scores.csv')

    return results, last_timestamps, mae_scores

if __name__ == "__main__":
    results, last_timestamps, mae_scores = main()


Processing A9...
Training LSTM...


epoch,train_loss,valid_loss,_rmse,time
0,0.772932,0.645551,0.803462,00:01
1,0.404186,0.018955,0.137676,00:01
2,0.185979,0.000040,0.006306,00:01
3,0.093260,0.000031,0.005605,00:01
4,0.048732,0.000025,0.004997,00:01
5,0.025992,0.000021,0.004551,00:01
6,0.014014,0.000018,0.004292,00:01
7,0.007601,0.000017,0.004092,00:01
8,0.004137,0.000016,0.003943,00:01
9,0.002258,0.000015,0.003933,00:01


LSTM prediction: 1.00, MAE: 0.0031
Training GRU...


epoch,train_loss,valid_loss,_rmse,time
0,0.775731,0.462476,0.680056,00:01
1,0.331836,0.010873,0.104276,00:01
2,0.152538,0.000084,0.009182,00:02
3,0.076484,0.000010,0.003168,00:01
4,0.039960,0.000005,0.002322,00:01
5,0.021308,0.000004,0.002046,00:01
6,0.011484,0.000004,0.001894,00:01
7,0.006224,0.000003,0.001832,00:01
8,0.003385,0.000003,0.001813,00:01
9,0.001844,0.000003,0.001811,00:01


GRU prediction: 1.00, MAE: 0.0015
Training MLP...


...

MLP prediction: 1.00, MAE: 0.0211
Training FCN...


epoch,train_loss,valid_loss,_rmse,time
0,0.422263,0.225959,0.475351,00:02
1,0.169185,0.002687,0.051835,00:03
2,0.079341,0.001732,0.041618,00:03
3,0.040614,0.000654,0.025569,00:02
4,0.021742,0.000671,0.025912,00:02
5,0.011931,0.001579,0.039742,00:02
6,0.006625,0.000396,0.019894,00:03
7,0.003746,0.000414,0.020353,00:02
8,0.002132,0.000667,0.025835,00:02
9,0.001236,0.000480,0.021909,00:02


FCN prediction: 0.98, MAE: 0.0173
Training ResNet...


epoch,train_loss,valid_loss,_rmse,time
0,0.636362,0.607668,0.779531,00:06
1,0.235136,0.022369,0.149564,00:04
2,0.109895,0.004983,0.070590,00:04
3,0.056414,0.002353,0.048510,00:06
4,0.030259,0.001833,0.042817,00:04
5,0.016686,0.001505,0.038790,00:05
6,0.009328,0.001593,0.039911,00:05
7,0.005303,0.001493,0.038641,00:05
8,0.003073,0.001806,0.042492,00:06
9,0.001815,0.001556,0.039443,00:04


ResNet prediction: 1.01, MAE: 0.0301

Last timestamps for each API:
A9: 2025-07-06 09:06:00
A2: 2025-07-06 09:45:00
A7: 2025-07-06 07:55:00
A4: 2025-07-06 10:22:00

Predicted number of events for next occurrence:
          A9    A2    A7    A4
LSTM    1.00  1.00  1.00  1.00
GRU     1.00  1.00  1.00  1.00
MLP     0.99  0.96  1.01  1.00
FCN     1.00  0.95  1.00  0.98
ResNet  1.00  1.01  1.00  1.01

MAE scores for each model:
            A9      A2      A7      A4
LSTM    0.0031  0.0010  0.0012  0.0013
GRU     0.0015  0.0008  0.0019  0.0011
MLP     0.0269  0.0163  0.0264  0.0211
FCN     0.0184  0.0239  0.0182  0.0173
ResNet  0.0293  0.0240  0.0269  0.0301


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from tsai.all import *
from fastai.tabular.all import *
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

def get_models(c_in=2, c_out=1, seq_len=24):
    """Return dictionary of all five models using direct PyTorch implementations"""

    # LSTM - using nn.LSTM directly
    lstm = nn.Sequential(
        nn.LSTM(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )

    # GRU - using nn.GRU directly
    gru = nn.Sequential(
        nn.GRU(input_size=c_in, hidden_size=64, num_layers=2, batch_first=True),
        nn.Linear(64, c_out)
    )

    # MLP - using nn.Sequential directly
    mlp = nn.Sequential(
        nn.Flatten(),
        nn.Linear(seq_len * c_in, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, c_out)
    )

    # FCN and ResNet from tsai
    fcn = FCN(c_in=c_in, c_out=c_out)
    resnet = ResNet(c_in=c_in, c_out=c_out)

    return {
        'LSTM': lstm,
        'GRU': gru,
        'MLP': mlp,
        'FCN': fcn,
        'ResNet': resnet
    }

def model_forward(model, x, model_name):
    """Handle forward pass for all model types"""
    if model_name in ['LSTM', 'GRU']:
        x = x.transpose(1, 2)  # Change to (batch, seq_len, features)
        if isinstance(model, nn.Sequential):
            output, _ = model[0](x)
            return model[1](output[:, -1, :])
    elif model_name == 'MLP':
        # MLP expects flattened input
        return model(x)
    else:
        # FCN and ResNet expect (batch, features, seq_len)
        return model(x)

def prepare_data(df, seq_length=24, split_ratio=0.8):
    """Prepare data for models with correct dimensions"""
    # Sort by timestamp to ensure temporal order
    df = df.sort_values('Time of Call')

    # Calculate time differences in seconds and normalize
    time_diffs = df['Time of Call'].diff().dt.total_seconds()
    time_diffs = time_diffs.fillna(time_diffs.mean())

    # Normalize time differences
    time_diffs = (time_diffs - time_diffs.mean()) / time_diffs.std()

    # Create sequences
    X, y = [], []
    for i in range(len(df) - seq_length):
        # Create sequence with 2 features: time_diff and count
        sequence = np.column_stack([
            time_diffs.iloc[i:i+seq_length],
            np.ones(seq_length)  # Event indicator
        ])
        X.append(sequence)
        y.append([1])  # Predicting next event

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)

    # For FCN and ResNet, shape should be (n_samples, n_features, seq_length)
    X = X.transpose(0, 2, 1)

    # Convert to tensors
    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)

    # Create splits
    train_size = int(len(X) * split_ratio)
    splits = (list(range(train_size)), list(range(train_size, len(X))))

    # Create TSai datasets
    tfms = [None, [TSRegression()]]
    dsets = TSDatasets(X, y, splits=splits, tfms=tfms)

    batch_size = min(64, len(X))
    dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, batch_size=batch_size)

    return dls, df['Time of Call'].iloc[-1]

def main():
    # Load the original dataset
    original_file = "API Call Dataset.csv"  # Update path as needed
    df_original = pd.read_csv(original_file)

    # Get the top 4 APIs
    api_counts = df_original['API Code'].value_counts().nlargest(4)
    top_apis = api_counts.index

    # Dictionary to store results
    last_timestamps = {}

    for api in top_apis:
        print(f"\nProcessing {api}...")

        # Load API-specific data
        file_path = f"{api}_data.csv"
        df = pd.read_csv(file_path)

        # Convert timestamp to datetime
        df['Time of Call'] = pd.to_datetime(df['Time of Call'], format='%d-%m-%Y %H:%M')

        # Prepare data
        dls, last_timestamp = prepare_data(df)
        last_timestamps[api] = last_timestamp

        # Get models
        models = get_models()

        # Train each model
        for model_name, model in models.items():
            print(f"Training {model_name}...")

            # Create learner with custom forward pass
            class ModelWrapper(nn.Module):
                def __init__(self, base_model, model_name):
                    super().__init__()
                    self.base_model = base_model
                    self.model_name = model_name

                def forward(self, x):
                    return model_forward(self.base_model, x, self.model_name)

            wrapped_model = ModelWrapper(model, model_name)
            learn = Learner(dls, wrapped_model, metrics=[rmse])

            # Train model
            learn.fit_one_cycle(10, lr_max=1e-3)

    # Print last timestamps
    print("\nLast timestamps for each API:")
    for api, timestamp in last_timestamps.items():
        print(f"{api}: {timestamp}")

    return last_timestamps

if __name__ == "__main__":
    last_timestamps = main()


Processing A9...
Training LSTM...


epoch,train_loss,valid_loss,_rmse,time
0,0.772932,0.645551,0.803462,00:01
1,0.404186,0.018955,0.137676,00:01
2,0.185979,0.000040,0.006306,00:01
3,0.093260,0.000031,0.005605,00:01
4,0.048732,0.000025,0.004997,00:01
5,0.025992,0.000021,0.004551,00:01
6,0.014014,0.000018,0.004292,00:01
7,0.007601,0.000017,0.004092,00:01
8,0.004137,0.000016,0.003943,00:01
9,0.002258,0.000015,0.003933,00:01


Training GRU...


epoch,train_loss,valid_loss,_rmse,time
0,0.775731,0.462476,0.680056,00:01
1,0.331836,0.010873,0.104276,00:01
2,0.152538,0.000084,0.009182,00:01
3,0.076484,0.000010,0.003168,00:01
4,0.039960,0.000005,0.002322,00:01
5,0.021308,0.000004,0.002046,00:01
6,0.011484,0.000004,0.001894,00:01
7,0.006224,0.000003,0.001832,00:01
8,0.003385,0.000003,0.001813,00:01
9,0.001844,0.000003,0.001811,00:01


Training MLP...


...

Training FCN...


epoch,train_loss,valid_loss,_rmse,time
0,0.414246,0.263163,0.512995,00:03
1,0.164189,0.003026,0.055010,00:02
2,0.077036,0.000591,0.024319,00:02
3,0.039445,0.000945,0.030746,00:02
4,0.021149,0.000756,0.027496,00:03
5,0.011613,0.001445,0.038014,00:03
6,0.006454,0.000412,0.020290,00:02
7,0.003644,0.000326,0.018067,00:02
8,0.002071,0.000574,0.023957,00:02
9,0.001204,0.000363,0.019060,00:02


Training ResNet...


epoch,train_loss,valid_loss,_rmse,time
0,0.875658,0.778591,0.882378,00:05
1,0.321951,0.009011,0.094927,00:04
2,0.150158,0.002807,0.052984,00:05
3,0.076834,0.002776,0.052686,00:05
4,0.041090,0.002033,0.045087,00:04
5,0.022524,0.001685,0.041054,00:05
6,0.012528,0.001537,0.039208,00:04
7,0.007062,0.001492,0.038624,00:04
8,0.004044,0.001997,0.044684,00:05
9,0.002355,0.001697,0.041192,00:04



Last timestamps for each API:
A9: 2025-07-06 09:06:00
A2: 2025-07-06 09:45:00
A7: 2025-07-06 07:55:00
A4: 2025-07-06 10:22:00
